In [1]:
import os
os.chdir("/Users/travishong/aml-lsc-hierarchy")

import pandas as pd

df = pd.read_csv("data/Fig3_Relapse_Deconvolution/CIBERSORTx_DxRx_Christopher_2018.csv")
print(df.head())
print(df.columns)


            Mixture         B       CTL  GMP-like  LSPC-Cycle  LSPC-Primed  \
0    814916.primary  0.220441  0.053001  0.068790    0.207394     0.862701   
1  814916.post_allo  0.053377  0.000000  0.299619    1.043583     0.541941   
2    619751.primary  0.056299  0.246683  1.707088    0.150530     0.000000   
3  619751.post_allo  0.133104  0.027650  2.168135    0.030825     0.310435   
4    593890.primary  0.076167  0.000000  1.705329    0.301714     0.408239   

   LSPC-Quiescent  Mono-like  Monocyte        NK    Plasma  ProMono-like  \
0        0.837905   0.140699  0.246259  0.025172  0.023996       0.00000   
1        0.109628   0.262025  0.200887  0.090274  0.035758       0.00000   
2        0.238252   1.223110  0.093103  0.053569  0.122587       0.00000   
3        0.000000   0.000000  0.110881  0.004989  0.058770       0.00000   
4        0.000000   0.265864  0.222013  0.399456  0.090199       0.05401   

          T       cDC  cDC-like  P-value  Correlation      RMSE  \
0  0.01

In [2]:
parts = df["Mixture"].str.split(".", n=1, expand=True)
df["patient_id"] = parts[0]
df["timepoint_raw"] = parts[1]

df["timepoint"] = df["timepoint_raw"].map({
    "primary": "diagnosis",
    "post_allo": "post_allo", 
    "post_chemo"    : "post_chemo",
})

df.head()

,Mixture,B,CTL,GMP-like,LSPC-Cycle,LSPC-Primed,LSPC-Quiescent,Mono-like,Monocyte,NK,...,T,cDC,cDC-like,P-value,Correlation,RMSE,Absolute score (sig.score),patient_id,timepoint_raw,timepoint
0,814916.primary,0.220441,0.053001,0.068790,0.207394,0.862701,0.837905,0.140699,0.246259,0.025172,...,0.015264,0.000000,0.718888,0,0.701654,0.880149,3.420511,814916,primary,diagnosis
1,814916.post_allo,0.053377,0.000000,0.299619,1.043583,0.541941,0.109628,0.262025,0.200887,0.090274,...,0.383245,0.521642,0.077950,0,0.880755,0.759798,3.619928,814916,post_allo,post_allo
2,619751.primary,0.056299,0.246683,1.707088,0.150530,0.000000,0.238252,1.223110,0.093103,0.053569,...,0.088911,0.125595,0.398440,0,0.947906,0.507869,4.504169,619751,primary,diagnosis
3,619751.post_allo,0.133104,0.027650,2.168135,0.030825,0.310435,0.000000,0.000000,0.110881,0.004989,...,0.019706,0.077992,0.385473,0,0.958835,0.552935,3.327962,619751,post_allo,post_allo
4,593890.primary,0.076167,0.000000,1.705329,0.301714,0.408239,0.000000,0.265864,0.222013,0.399456,...,0.507934,0.567208,0.461127,0,0.807929,0.724110,5.059260,593890,primary,diagnosis


In [3]:
primitive_cols = ["LSPC-Quiescent", "LSPC-Primed", "LSPC-Cycle"]
df["frac_Primitive"] = df[primitive_cols].sum(axis=1)

# sanity: view a few samples
df[["Mixture", "frac_Primitive"]].head()


,Mixture,frac_Primitive
0,814916.primary,1.908000
1,814916.post_allo,1.695152
2,619751.primary,0.388782
3,619751.post_allo,0.341261
4,593890.primary,0.709952


In [5]:
wide = df.pivot_table(
    index="patient_id",
    columns="timepoint",
    values="frac_Primitive"
)

wide["delta_prim_post_chemo"] = wide["post_chemo"] - wide["diagnosis"]
wide["delta_prim_post_allo"] = wide["post_allo"] - wide["diagnosis"]

wide.head()

timepoint,diagnosis,post_allo,post_chemo,delta_prim_post_chemo,delta_prim_post_allo
patient_id,,,,,
103342,0.573910,0.718412,NaN,NaN,0.144503
112200,0.158583,NaN,0.293180,0.134597,NaN
142074,2.001125,1.707914,NaN,NaN,-0.293211
150288,1.971424,NaN,2.024045,0.052621,NaN
196371,0.453170,NaN,1.545906,1.092736,NaN
